In [155]:
import scipy as sp
import matplotlib.pyplot as plt
import json
import pandas as pd
import os
from collections import OrderedDict
from IPython.display import display

from scipy import integrate, special, interpolate
%matplotlib inline

pd.set_option('precision', 10)

In [156]:
def norm(pname, outdirname):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname), index_col=0)
    return sp.sqrt(sp.integrate.trapz(points["u"]**2, points.index.values))

In [157]:
def plot(ax, pname, outdirname):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname), index_col=0)
    norm = sp.sqrt(sp.integrate.trapz(points["u"]**2, points.index.values))
    f = sp.interpolate.interp1d(points.index.values, points["u"])
    ax.plot(points.index.values*0.19732697, points["u"]**2/norm**2/0.19732697, ".")

In [158]:
def overlap(pname1, pname2, outdirname, func = lambda r: r):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points1 = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname1), index_col=0)
    points2 = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname2), index_col=0)
    f1 = sp.interpolate.interp1d(points1.index.values, points1["u"])
    f2 = sp.interpolate.interp1d(points2.index.values, points2["u"])
    if (points1.index.values[-1] > points2.index.values[-1]):
        xes = points2.index.values
    else:
        xes = points1.index.values
    return sp.integrate.trapz(f1(xes)*f2(xes)*func(xes), xes)/sp.sqrt(sp.integrate.trapz(points1["u"]**2, points1.index.values)*sp.integrate.trapz(points2["u"]**2, points2.index.values))

In [315]:
def readCSV(outdirname):
    data = pd.read_csv(open(os.path.join("output", outdirname, "data")), index_col=[0,1])
    cfg = json.load(open(os.path.join("output", outdirname, "eigen_config"), "r"), object_pairs_hook=OrderedDict)
    def k(row):
        Mf = 2*cfg[row.name[1]]["eq"]["env"]["mC"]+cfg[row.name[1]]["eq"]["E"]
        return -Mf+sp.sqrt(Mf**2 + 2*Mf*(cfg[row.name[0]]["eq"]["E"] - cfg[row.name[1]]["eq"]["E"]))
    lap1 = data.apply(lambda r: overlap(*r.name, outdirname), axis=1).rename("melLW")
    lapful = data.apply(lambda r: overlap(*r.name, outdirname, lambda x: x*(sp.special.spherical_jn(0, k(r)*x/2) - sp.sqrt(5/2)*sp.special.spherical_jn(2, k(r)*x/2))), axis=1).rename("melE1")
    gamma = data.apply(k, axis=1).rename("$E_\gamma$")
    data = data.join(lap1).join(lapful).join(gamma)
    check = ((data["widthE1"]/data["melE1"]**2-data["widthELW"]/data["melLW"]**2)*2/(data["widthE1"]/data["melE1"]**2+data["widthELW"]/data["melLW"]**2)).rename("check")
    data = data.join(check)
    return data

In [218]:
def assembleData(datapath, paperpath):
    chin = pd.read_csv(paperpath, index_col=[0,1]).rename(columns={"width": "widthPAPER"})
    report = readCSV(datapath)
    report = pd.concat([report, chin], axis=1)
    
    return report

In [581]:
def relativeQuant(report):
    return pd.DataFrame(index=report.index)\
        .join(\
            (report["widthPAPER"]/report["widthE1"])\
                .rename("PAPER/E1"))\
        .join(\
              (report["widthPAPER"]/report["width"])\
                  .rename("PAPER/width"))\
        .join(\
              (report["widthPAPER"]/report["width_mod"])\
                  .rename("PAPER/width_mod"))
        .join(\
              (report["widthPAPER"]/report["widthE1_mod"])\
                  .rename("PAPER/widthE1_mod"))\
        .join(\
              (report["widthPAPER"]/report["widthELW_mod"])\
                  .rename("PAPER/widthELW_mod")\
             )

In [576]:
def experimentalReport(report, transdata):
    trans = pd.read_csv(transdata, index_col=[0, 1]).add_suffix("_pdg")
    pdata = pd.read_csv("../data/bottomonia.csv", index_col=0)
    width = trans.apply(lambda r: r["frac_width_pdg"]*pdata.loc[r.name[0], "PDG_width"],axis=1).rename("width_pdg")
    
    eMix = report["$E_\gamma$"].copy()
    notnanE = sp.logical_not(sp.isnan(trans["egamma_pdg"]))
    notnanE = [i for i in notnanE[notnanE].index.values for j in report.index.values if i == j]
    eMix[notnanE] = trans["egamma_pdg"][notnanE].copy()
    eMix.rename("E", inplace=True)
    
    MMix = pdata["PDG"].copy()
    nanM = sp.isnan(MMix)
    nanM = nanM[nanM].index.values
    MMix[nanM] = pdata["SP"][nanM].copy()
    MMix /= 1000
    
    expfactor = eMix.index.map(\
              lambda row: eMix[row]*MMix[row[0]]/(eMix[row] + MMix[row[0]])*(MMix[row[0]] - MMix[row[1]])**2\
        )
    
    width_mod = (expfactor*report["R_width"].copy()).rename("width_mod")
    widthE1_mod = (expfactor*report["R_widthE1"].copy()).rename("widthE1_mod")
    widthELW_mod = (expfactor*report["R_widthELW"].copy()).rename("widthELW_mod")
    
    return pd.concat([report[["width", "widthE1", "widthELW", "$E_\gamma$", "widthPAPER"]], width_mod, widthE1_mod, widthELW_mod, trans[["egamma_pdg"]]], join="inner", axis=1).join(width)

In [598]:
def doReport(datapath, paperpath, transdata):
    data = assembleData(datapath, paperpath)
    exper = experimentalReport(data, transdata)
    rel = (relativeQuant(exper)-1.)*100
    rel.rename(columns={i: i+"(%)" for i in rel.columns}, inplace=True)
    
    labels = ["width", "widthE1", "widthELW", "$E_\gamma$", "widthPAPER"]
    data[labels] *= 10**6
    data.rename(columns=dict(zip(labels, ["%s(KeV)"%i for i in labels])), inplace=True)
    
    labels = ["melLW", "melE1"]
    data[labels] *= 10**(-6)
    data.rename(columns=dict(zip(labels, ["%s($KeV^{-1}$)"%i for i in labels])), inplace=True)
    
    labels = ["R_width", "R_widthE1", "R_widthELW"]
    data[labels] *= 10**(-12)
    data.rename(columns=dict(zip(labels, ["%s($KeV^{-2}$)"%i for i in labels])), inplace=True)
    
    labels = ["width", "widthE1", "widthELW", "width_mod", "widthE1_mod", "widthELW_mod", "$E_\gamma$", "widthPAPER", "egamma_pdg", "width_pdg"]
    exper[labels] *= 10**6
    exper.rename(columns=dict(zip(labels, ["%s(KeV)"%i for i in labels])), inplace=True)
    
    display(data)
    display(exper)
    display(rel)

In [599]:
doReport("scr.E1.1P1D2D.b-scr", "../data/deng2017-1P1D2D.csv", "../data/bottom-trans-PDG.csv")

width(KeV)  R_width($KeV^{-2}$)   widthE1(KeV)  \
instate   outstate                                                       
chi_b2_1P yps_1S     32.7811253634     3.5886480896e-16  34.3642223127   
chi_b1_1P yps_1S     33.5519843831     4.1283528207e-16  32.0530633801   
chi_b0_1P yps_1S     28.2994845483     4.5647987262e-16  26.0843069433   
h_b_1P    eta_b_1S   38.5416696992     2.9835217333e-16  38.5416696992   
yps_3_1D  chi_b2_1P  21.9390761004     1.7284258460e-15  22.4851517410   
          chi_b1_1P   0.0046102348     2.9211247830e-19   0.0000000000   
          chi_b0_1P   0.0019160756     7.9552236400e-20   0.0000000000   
yps_2_1D  chi_b2_1P   5.6496242867     4.6824403710e-16   5.3839108405   
          chi_b1_1P  18.3217602689     1.2168133385e-15  18.3180461172   
          chi_b0_1P   0.0041883034     1.8110756160e-19   0.0000000000   
yps_1_1D  chi_b2_1P   0.6065495532     5.5054690683e-17   0.5510390615   
          chi_b1_1P   9.9529286242     7.1906548845e-16   9.4655746921   
          chi_b0_1P  16.6313371020     7.7327286115e-16  16.1518196876   
eta_b2_1D h_b_1P     23.4365989593     1.6726887500e-15  23.4333907808   
yps_3_2D  chi_b2_1P   6.6115800905     5.2142245184e-17   6.8659618164   
          chi_b1_1P   0.0407565407     2.9070849160e-19   0.0000000000   
          chi_b0_1P   0.0136953682     7.9485980800e-20   0.0000000000   
          chi_b2_2P  15.7187526738     3.1665411740e-15  16.0062222486   
          chi_b1_2P   0.0039098090     6.1421784190e-19   0.0000000000   
          chi_b0_2P   0.0029860496     3.0566395170e-19   0.0000000000   
yps_2_2D  chi_b2_1P   1.5721252158     1.2685003693e-17   1.3914575341   
          chi_b1_1P   6.2748592601     4.5755082452e-17   6.2436266336   
          chi_b0_1P   0.0057745125     3.4208676400e-20   0.0000000000   
          chi_b2_2P   4.0028301571     8.6485901318e-16   3.8621748241   
          chi_b1_2P  13.0779684378     2.1909506909e-15  13.0752332698   
          chi_b0_2P   0.0015454380     1.6722719070e-19   0.0000000000   
yps_1_2D  chi_b2_1P   0.2339356541     1.9653984494e-18   0.1114910091   
          chi_b1_1P   2.9182414507     2.2125458982e-17   2.6860406156   
          chi_b0_1P   7.2241778581     4.4375906208e-17   6.8833495727   
          chi_b2_2P   0.4242433353     1.0404274091e-16   0.3943072221   
          chi_b1_2P   7.0537997637     1.3270876429e-15   6.7977543704   
          chi_b0_2P  11.1175115099     1.3291646365e-15  10.8786877559   
eta_b2_2D h_b_1P      7.7112738590     5.8135931675e-17   7.6225761489   
          h_b_2P     16.6298366242     3.0252435024e-15  16.6253587072   

                     R_widthE1($KeV^{-2}$)  widthELW(KeV)  \
instate   outstate                                          
chi_b2_1P yps_1S          3.7619544596e-16  37.8252906937   
chi_b1_1P yps_1S          3.9439203687e-16  34.8975200441   
chi_b0_1P yps_1S          4.2074833874e-16  27.8527539278   
h_b_1P    eta_b_1S        2.9835217333e-16  43.0249501811   
yps_3_1D  chi_b2_1P       1.7714473135e-15  23.7512643477   
          chi_b1_1P       0.0000000000e+00   0.0000000000   
          chi_b0_1P       0.0000000000e+00   0.0000000000   
yps_2_1D  chi_b2_1P       4.4622155729e-16   5.6729780773   
          chi_b1_1P       1.2165666684e-15  19.4331548358   
          chi_b0_1P       0.0000000000e+00   0.0000000000   
yps_1_1D  chi_b2_1P       5.0016169203e-17   0.5783914020   
          chi_b1_1P       6.8385581234e-16  10.0003462001   
          chi_b0_1P       7.5097773234e-16  17.3270538766   
eta_b2_1D h_b_1P          1.6724597797e-15  24.8074784774   
yps_3_2D  chi_b2_1P       5.4148427389e-17   3.3454719362   
          chi_b1_1P       0.0000000000e+00   0.0000000000   
          chi_b0_1P       0.0000000000e+00   0.0000000000   
          chi_b2_2P       3.2244518915e-15  17.2542851712   
          chi_b1_2P       0.0000000000e+00   0.0000000000   
          chi_b0_2P       0.0000000000e+00   0.0000000000   
yps_2_2D  chi_b2_

,,width(KeV),widthE1(KeV),widthELW(KeV),$E_\gamma$(KeV),widthPAPER(KeV),width_mod(KeV),widthE1_mod(KeV),widthELW_mod(KeV),egamma_pdg(KeV),width_pdg(KeV)
instate,outstate,,,,,,,,,,
chi_b0_1P,yps_1S,28.2994845483,26.0843069433,27.8527539278,395723.2103991242,27.5,27.6482642879,25.4840617647,27.2118137144,NaN,NaN
chi_b1_1P,yps_1S,33.5519843831,32.0530633801,34.8975200441,433086.4749076166,31.9,32.1158537438,30.6810912822,33.4037962393,NaN,NaN
chi_b2_1P,yps_1S,32.7811253634,34.3642223127,37.8252906937,450283.5479112300,31.8,31.5790923292,33.1041395682,36.4382959387,NaN,NaN
h_b_1P,eta_b_1S,38.5416696992,38.5416696992,43.0249501811,505398.9308162805,35.8,36.0091860590,36.0091860590,40.1978805885,NaN,NaN


,,PAPER/E1(%),PAPER/width(%),PAPER/width_mod(%),PAPER/widthE1_mod(%),PAPER/widthELW_mod(%)
instate,outstate,,,,,
chi_b0_1P,yps_1S,5.4273746271,-2.8250851952,-0.5362517024,7.9105844819,1.0590484288
chi_b1_1P,yps_1S,-0.4775312059,-4.9236562709,-0.6721096240,3.9728336471,-4.5018722679
chi_b2_1P,yps_1S,-7.4618953671,-2.9929581505,0.6995377463,-3.9395060110,-12.7291790662
h_b_1P,eta_b_1S,-7.1135208220,-7.1135208220,-0.5809241526,-0.5809241526,-10.9405782697


In [600]:
doReport("scr.E1.nSnP.b-scr", "../data/deng2017-nSnP.csv", "../data/bottom-trans-PDG.csv")

width(KeV)  R_width($KeV^{-2}$)   widthE1(KeV)  \
instate   outstate                                                       
yps_2S    chi_b2_1P   1.5938352131     1.9461629466e-15   1.5782368885   
          chi_b1_1P   1.3981408191     1.0120201515e-15   1.4145780448   
          chi_b0_1P   0.8609248565     2.5578173504e-16   0.8886924283   
eta_b_2S  h_b_1P      1.8226266730     3.4715890486e-15   1.8226266730   
yps_3S    chi_b2_2P   2.4418443892     5.0097458791e-15   2.4216767628   
          chi_b1_2P   2.1541683652     2.6291853877e-15   2.1755082396   
          chi_b0_2P   1.2280721112     6.7183649036e-16   1.2604498621   
          chi_b2_1P   0.0003452105     4.8846224000e-21   0.0000406132   
          chi_b1_1P   0.0840900798     1.0524820796e-18   0.0839756328   
          chi_b0_1P   0.2087393820     2.0365978968e-18   0.2221871683   
eta_b_3S  h_b_2P      3.3324876315     9.0222752288e-15   3.3324876315   
          h_b_1P      0.1064173806     1.5596057694e-18   0.1064173806   
chi_b2_2P yps_3_1D    2.6212855538     2.1735032177e-15   2.5920255616   
          yps_2_1D    0.4745102200     3.5263732077e-16   0.4857112049   
          yps_1_1D    0.0341068139     2.1119903551e-17   0.0358021930   
          yps_2S     15.3808260019     1.0332855640e-15  15.7910710988   
          yps_1S     16.8425044756     3.6484333572e-17  18.0796624327   
chi_b1_2P yps_2_1D    1.7575619347     2.0146693702e-15   1.7575156806   
          yps_1_1D    0.6560968117     6.0954923479e-16   0.6704728249   
          yps_2S     15.4302441048     1.2463198333e-15  15.0528184918   
          yps_1S     14.9664929227     3.4234295677e-17  13.9691126260   
chi_b0_2P yps_1_1D    1.1844247406     2.9548275321e-15   1.1936523864   
          yps_2S     13.0098344610     1.5563270570e-15  12.4663411580   
          yps_1S      7.9654078909     2.0304948577e-17   7.0651249821   
h_b_2P    eta_b2_1D   2.6702726830     2.6303500366e-15   2.6702266099   
          eta_b_2S   16.2106648131     9.1580034648e-16  16.2106648131   
          eta_b_1S   21.4614076814     3.7802883263e-17  21.4614076814   
chi_b2_3P yps_3_2D    3.8952833372     5.0684194811e-15   3.8576800106   
          yps_2_2D    0.7197974387     8.2478636850e-16   0.7344723322   
          yps_1_2D    0.0531746086     4.9382305897e-17   0.0554033439   
          yps_3_1D    0.4259817080     8.7963517614e-18   0.4053127880   
          yps_2_1D    0.1105903736     2.2126166263e-18   0.0929048311   
          yps_1_1D    0.0259179825     4.9105490380e-19   0.0083418074   
          yps_3S     12.0737731853     1.9579113727e-15  12.3153523538   
          yps_2S      9.2256392929     7.3543035968e-17   9.6485498547   
          yps_1S     11.6520757857     1.1157441605e-17  12.7054831720   
chi_b1_3P yps_2_2D    2.6733400851     4.7311374829e-15   2.6732676543   
          yps_1_2D    1.0302597907     1.4315257229e-15   1.0500336486   
          yps_2_1D    0.1826453088     4.0549290219e-18   0.1647498525   
          yps_1_1D    0.0907076880     1.9032777087e-18   0.0935141053   
          yps_3S     11.8972691076     2.3962572282e-15  11.6805760354   
          yps_2S      8.1789820395     7.0293039781e-17   7.8369805742   
          yps_1S      9.6106257783     9.5307243044e-18   8.8388043179   
chi_b0_3P yps_1_2D    1.9021553676     6.9910857990e-15   1.9151959243   
          yps_1_1D    0.0000329824     8.5120760000e-22   0.0001441574   
          yps_3S      9.6369945743     3.0983448092e-15   9.3436304842   
          yps_2S      4.5439324098     4.5392149367e-17   4.2266139385   
          yps_1S      3.7097905952     3.9403544113e-18   3.2370350597   
h_b_3P    eta_b2_2D   4.0180516911     6.1780228269e-15   4.0179194209   
          eta_b2_1D   0.3563840016     7.6597791916e-18   0.3009143543   
          eta_b_3S   12.5350455854     1.7923635445e-15  12.5350455854   
          eta_b_2S   10.3139197045     7.5159325852e-17  10.3139197045   

                     R_widt

width(KeV)   widthE1(KeV)  widthELW(KeV)  \
instate   outstate                                                 
chi_b0_2P yps_1S      7.9654078909   7.0651249821   3.7873128891   
          yps_2S     13.0098344610  12.4663411580  13.3215731725   
chi_b1_2P yps_1S     14.9664929227  13.9691126260   9.7906464121   
          yps_2S     15.4302441048  15.0528184918  16.5389647765   
chi_b2_2P yps_1S     16.8425044756  18.0796624327  13.8335924125   
          yps_2S     15.3808260019  15.7910710988  17.6683641056   
h_b_2P    eta_b_1S   21.4614076814  21.4614076814  17.3263935110   
          eta_b_2S   16.2106648131  16.2106648131  18.3367058315   
yps_2S    chi_b0_1P   0.8609248565   0.8886924283   0.9094124270   
          chi_b1_1P   1.3981408191   1.4145780448   1.4330281456   
          chi_b2_1P   1.5938352131   1.5782368885   1.5929031380   
yps_3S    chi_b0_1P   0.2087393820   0.2221871683   0.0393770605   
          chi_b0_2P   1.2280721112   1.2604498621   1.3086236137   
          chi_b1_1P   0.0840900798   0.0839756328   0.0356635715   
          chi_b1_2P   2.1541683652   2.1755082396   2.2232695552   
          chi_b2_1P   0.0003452105   0.0000406132   0.3941748934   
          chi_b2_2P   2.4418443892   2.4216767628   2.4590729012   

                       $E_\gamma$(KeV)  widthPAPER(KeV)  width_mod(KeV)  \
instate   outstate                                                        
chi_b0_2P yps_1S     731702.5074322104           5.5400    8.2851723332   
          yps_2S     202943.7755352710          14.4000   13.6579788678   
chi_b1_2P yps_1S     758585.0292494029          10.8000   15.2829389352   
          yps_2S     231328.3667858315          15.3000   15.1756247574   
chi_b2_2P yps_1S     772462.0693164539          12.5000   17.1546630926   
          yps_2S     245979.2417683016          15.3000   15.0213431999   
h_b_2P    eta_b_1S   827534.9277602171          16.1000   21.5098880334   
          eta_b_2S   260600.9579926578          16.2000   15.8549413022   
yps_2S    chi_b0_1P  149861.5928122344           1.0900    1.1000843026   
          chi_b1_1P  111373.7198089417           2.1700    2.1887721655   
          chi_b2_1P   93558.8519914354           2.6200    2.6193430764   
yps_3S    chi_b0_1P  467902.4385637707           0.0970    0.2243005364   
          chi_b0_2P  122268.6099327355           1.2100    1.2047703203   
          chi_b1_1P  430636.7751145572           0.0005    0.0928782638   
          chi_b1_2P   93572.8423830788           2.6100    2.5849508335   
          chi_b2_1P  413387.4999924405           0.1400    0.0003810616   
          chi_b2_2P   78698.2939656529           3.1600    3.1671569412   

                     widthE1_mod(KeV)  widthELW_mod(KeV)  egamma_pdg(KeV)  \
instate   outstate                                                          
chi_b0_2P yps_1S         7.3487483421       3.9393513045              NaN   
          yps_2S        13.0874089601      13.9852482690              NaN   
chi_b1_2P yps_1S        14.2644703969       9.9976562329              NaN   
          yps_2S        14.8044271640      16.2660500780              NaN   
chi_b2_2P yps_1S        18.4147505088      14.0899839178              NaN   
          yps_2S        15.4219999914      17.2554166453              NaN   
h_b_2P    eta_b_1S      21.5098880334      17.3655330525              NaN   
          eta_b_2S      15.8549413022      17.9343289116              NaN   
yps_2S    chi_b0_1P      1.1355655291       1.1620414115         162500.0   
          chi_b1_1P      2.2145044391       2.2433878437         129630.0   
          chi_b2_1P      2.5937084542       2.6178112841         110440.0   
yps_3S    chi_b0_1P      0.2387508316       0.0423125512              NaN   
          chi_b0_2P      1.2365337266       1.2837934157         121900.0   
          chi_b1_1P      0.0927518560       0.0393907416              NaN   
          chi_b1_2P      2.6105581755       2.6678706189          99260.0

PAPER/E1(%)    PAPER/width(%)  PAPER/width_mod(%)  \
instate   outstate                                                             
chi_b0_2P yps_1S        -21.5866667039    -30.4492616589      -33.1335574297   
          yps_2S         15.5110374206     10.6854975224        5.4328765577   
chi_b1_2P yps_1S        -22.6865707998    -27.8388059529      -29.3329637331   
          yps_2S          1.6420945243     -0.8440832427        0.8195724697   
chi_b2_2P yps_1S        -30.8615409910    -25.7830091831      -27.1335150535   
          yps_2S         -3.1098023416     -0.5254984478        1.8550724552   
h_b_2P    eta_b_1S      -24.9816217137    -24.9816217137      -25.1507029001   
          eta_b_2S       -0.0657888694     -0.0657888694        2.1763479990   
yps_2S    chi_b0_1P      22.6521083436     26.6080299309       -0.9166845255   
          chi_b1_1P      53.4026353637     55.2061115987       -0.8576573561   
          chi_b2_1P      66.0080320699     64.3833677701        0.0250797091   
yps_3S    chi_b0_1P     -56.3431134470    -53.5305704795      -56.7544502760   
          chi_b0_2P      -4.0025282732     -1.4715838781        0.4340810501   
          chi_b1_1P     -99.4045891846    -99.4053995416      -99.4616609099   
          chi_b1_2P      19.9719657453     21.1604460526        0.9690384115   
          chi_b2_1P  344615.5112131031  40454.9657382959    36639.4641868794   
          chi_b2_2P      30.4881001685     29.4103757789       -0.2259736843   

                     PAPER/widthE1_mod(%)  PAPER/widthELW_mod(%)  
instate   outstate                                                
chi_b0_2P yps_1S           -24.6130124197          40.6322912508  
          yps_2S            10.0294186869           2.9656372416  
chi_b1_2P yps_1S           -24.2874099109           8.0253186192  
          yps_2S             3.3474637721          -5.9390575667  
chi_b2_2P yps_1S           -32.1196342353         -11.2844977470  
          yps_2S            -0.7910776258         -11.3321902653  
h_b_2P    eta_b_1S         -25.1507029001          -7.2876141993  
          eta_b_2S           2.1763479990          -9.6704422015  
yps_2S    chi_b0_1P         -4.0125847350          -6.1995563010  
          chi_b1_1P         -2.0096793816          -3.2712954181  
          chi_b2_1P          1.0136661938           0.0836086189  
yps_3S    chi_b0_1P        -59.3718692566         129.2463988784  
          chi_b0_2P         -2.1458150329          -5.7480755677  
          chi_b1_1P        -99.4609272294         -98.7306661933  
          chi_b1_2P         -0.0213814612          -2.1691688691  
          chi_b2_1P     312183.5755780962         -67.8243146624  
          chi_b2_2P          0.6049403818          -0.9250005475